In [1]:
!pip uninstall accelerate -y


In [2]:
!pip install accelerate==0.20.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 3.3 MB/s eta 0:00:00


In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset, load_metric

In [7]:
dataset = load_dataset("glue", "mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [9]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

,premise,hypothesis,label,idx
0,"One of the best beaches, pockmarked with volcanic steam spouts, is the Lido dei Maronti on the south coast near the little fishing village of Sant'Angelo.",Lido dei Maronti is located near Sant'Angelo.,entailment,2578
1,Shows 8pm; dark Monday and Frida y; over $60.,The tickets sell out for every 8pm show.,neutral,387546
2,"The endless obstacles to confirmation deter the best According to Mackenzie, the presidential personnel office must frequently offer a job to its fourth or fifth choice because the top candidates don't want to endure the inconvenience.",The presidential personnel office only offers jobs to its first choice candidate.,contradiction,222856
3,"The bulk of the solid square structure stands around a vast courtyard, Piazza d'Armiles.",There are no buildings near Piazza d'Armiles.,contradiction,66879
4,"It is the rate used to credit interest to the dividend fund, and against which experience is measured to determine the amount of the interest portion of dividends paid to individual policyholders.",The amount of interest paid to policyholders is a function of a formula.,entailment,14001
5,"These traditional reasons, you say explain its significance for many [Jews] at a preconscious level, whether we like it or not.",We have no control over the reasons' explanation of its preconscious significance for many Jews.,entailment,274719
6,Could he? She appealed to Sir James.,She was ignoring Sir James.,contradiction,329312
7,"In fact, the timeline of major events in GAO's existence (see figure 4) reveals the increasing development, complexity and influence of difficult public policy issues related to government activity and our accountability",We have not received a copy of the timeline of GAO's major events.,contradiction,255810
8,Valuing Fatal Cancer Risk Reductions.,No evaluation of fatal cancer risk.,contradiction,383572
9,"It took a lot of hard work by all of the committee members to make this a reality, Bailey said.",The committee was formed to discuss research.,neutral,279174


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-large")

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
task_to_keys = {
    "mnli": ("premise", "hypothesis"),
}

In [14]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [15]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 9157, 16771, 13851, 6353, 2972, 16364, 34, 80, 3280, 22735, 111, 1152, 8, 18947, 4, 2, 2, 41257, 8, 18947, 32, 99, 146, 6353, 2972, 16364, 173, 4, 1437, 2], [0, 6968, 216, 148, 5, 191, 8, 939, 4443, 23, 23, 110, 672, 37463, 47, 2217, 106, 7, 5, 220, 672, 114, 114, 51, 2845, 7, 6001, 5, 5, 4095, 165, 5, 13376, 2845, 7, 486, 7, 6001, 10, 2173, 31, 6436, 83, 172, 10, 1457, 83, 2173, 1411, 62, 7, 3190, 123, 8, 10, 881, 83, 2173, 1411, 62, 7, 3190, 123, 2, 2, 1185, 2217, 5, 383, 7, 5, 511, 672, 114, 5, 82, 6001, 4, 2], [0, 3762, 9, 84, 346, 40, 2324, 66, 110, 9223, 2289, 352, 4, 2, 2, 250, 919, 9, 127, 165, 40, 11189, 110, 3365, 19, 14151, 15339, 4, 2], [0, 6179, 109, 47, 216, 116, 404, 42, 16, 49, 335, 456, 4, 2, 2, 713, 335, 12918, 7, 106, 4, 2], [0, 42803, 939, 1137, 47, 99, 600, 114, 47, 213, 425, 103, 9, 167, 5919, 5582, 939, 64, 192, 596, 122, 47, 216, 51, 214, 562, 62, 11, 5, 6317, 1404, 1186, 2, 2, 133, 5919, 5582, 33, 10, 1186, 9, 850, 4, 2]], 'attention_mask': [

In [16]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
batch_size = 2
metric_name = "accuracy"


args = TrainingArguments(
    f"{'roberta-large'}-finetuned-{'mnli'}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [20]:
metric = load_metric('glue', "mnli")

<ipython-input-20-17bf5bf84e68>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [27]:
validation_key = "validation_matched"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 